In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import time
import math
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and process the dataset

In [ ]:
# train BPETokenizer 
def train_bpe_tokenizer(texts, vocab_size=26500, min_frequency=2, special_tokens=["[UNK]", "[SOS]", "[EOS]"]):
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    trainer = BpeTrainer(
        vocab_size=vocab_size,
        min_frequency=min_frequency,
        special_tokens=special_tokens
    )
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

# construct word2index, word2count and index2word dicts
def build_dicts(tokenizer, texts):
    word2index = tokenizer.get_vocab()
    word2count = {token: 0 for token in word2index.keys()}
    index2word = {index: token for token, index in word2index.items()}

    for text in texts:
        output = tokenizer.encode(text)
        tokens = output.tokens
        for token in tokens:
            word2count[token] += 1

    return word2index, word2count, index2word

def normalizeStringEng(s):
    s = re.sub(r'[^\w\s]', '', s)
    s = s.lower().strip()
    return s

def normalizeStringChi(s):
    s = re.sub(r'[^\u4e00-\u9fa5\s]', '', s)
    s = s.strip()
    return s

# read corpus
def read_corpus(file_path):
    english_texts = []
    chinese_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                english_texts.append(normalizeStringEng(parts[0]))
                chinese_texts.append(normalizeStringChi(parts[1]))
    return english_texts, chinese_texts


file_path = "data/raw/partA/eng-cmn.txt"  
english_texts, chinese_texts = read_corpus(file_path)

pairs = list(zip(english_texts, chinese_texts))
# train english tokenizer
english_tokenizer = train_bpe_tokenizer(english_texts)
english_word2index, english_word2count, english_index2word = build_dicts(english_tokenizer, english_texts)

# train chinese tokenizer
chinese_tokenizer = train_bpe_tokenizer(chinese_texts)
chinese_word2index, chinese_word2count, chinese_index2word = build_dicts(chinese_tokenizer, chinese_texts)


In [4]:
# print(english_texts[:5])
# print(chinese_texts[:5])
# print(pairs[:5])

In [5]:
# 输出部分结果示例
# print(english_word2index)
# print(len(english_word2index))
# print(english_index2word[2])
# print(len(english_index2word))
# sorted_index2word = {key: english_index2word[key] for key in sorted(english_index2word.keys())}
# print(sorted_index2word)


# print(english_word2count)

# print(chinese_word2index)
# print(len(chinese_word2index))
# print(chinese_index2word)

In [6]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

MAX_LENGTH = 10
pairs = filterPairs(pairs)
pairs = [list(reversed(p)) for p in pairs]


In [ ]:
UNK_token = 0
SOS_token = 1
EOS_token = 2


def indexesFromSentence(tokenizer, sentence):
    # print(sentence)
    encoded = tokenizer.encode(sentence)
    return encoded.ids

def index2sentence(indexes, tokenizer, skip_special_tokens=True):

    tokens = []
    for idx in indexes:
        token = tokenizer.id_to_token(idx)
        
        # handle special tokens
        if skip_special_tokens and token in ['[UNK]', '[SOS]', '[EOS]']:
            continue
        
        # handle subword tokens
        if token.startswith('##'):
            tokens[-1] += token[2:]
        else:
            tokens.append(token)
    
    return ''.join(tokens)

def tensorFromSentence(tokenizer, sentence):
    indexes = indexesFromSentence(tokenizer, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)


In [8]:
def get_dataloaders(batch_size):
    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        
        inp_ids = indexesFromSentence(chinese_tokenizer, inp )[:MAX_LENGTH-1]
        tgt_ids = indexesFromSentence(english_tokenizer, tgt)[:MAX_LENGTH-1]
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        # print(f"inp_ids: {inp_ids}")
        # print(f"tgt_ids: {tgt_ids}")
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    dataset = TensorDataset(torch.LongTensor(input_ids).to(device),
                            torch.LongTensor(target_ids).to(device))

    train_size = int(0.9 * n)

    indices = list(range(n))
    random.shuffle(indices)

    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    train_data = torch.utils.data.Subset(dataset, train_indices)
    test_data = torch.utils.data.Subset(dataset, test_indices)

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return train_dataloader, test_dataloader

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [10]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig('plot_s2s.png')
    plt.close()

In [12]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
hidden_size = 256
batch_size = 64

train_dataloader, test_dataloader = get_dataloaders(batch_size)

encoder = EncoderRNN(len(chinese_word2count), hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, len(english_word2count)).to(device)

train(train_dataloader, encoder, decoder, 128, print_every=5, plot_every=5)

0m 55s (- 22m 49s) (5 3%) 2.0359
1m 51s (- 21m 50s) (10 7%) 0.4135
2m 46s (- 20m 52s) (15 11%) 0.1495
3m 41s (- 19m 53s) (20 15%) 0.1276
4m 33s (- 18m 45s) (25 19%) 0.1176
5m 25s (- 17m 43s) (30 23%) 0.1101
6m 17s (- 16m 43s) (35 27%) 0.1068
7m 8s (- 15m 43s) (40 31%) 0.1034
8m 3s (- 14m 52s) (45 35%) 0.1016
8m 55s (- 13m 54s) (50 39%) 0.1009
9m 46s (- 12m 58s) (55 42%) 0.0998
10m 37s (- 12m 2s) (60 46%) 0.0985
11m 30s (- 11m 9s) (65 50%) 0.0977
12m 24s (- 10m 16s) (70 54%) 0.0970
13m 13s (- 9m 20s) (75 58%) 0.0969
14m 5s (- 8m 27s) (80 62%) 0.0966
14m 51s (- 7m 30s) (85 66%) 0.0962
15m 37s (- 6m 35s) (90 70%) 0.0964
16m 26s (- 5m 42s) (95 74%) 0.0960
17m 14s (- 4m 49s) (100 78%) 0.0963
18m 3s (- 3m 57s) (105 82%) 0.0976
18m 52s (- 3m 5s) (110 85%) 0.0978
19m 41s (- 2m 13s) (115 89%) 0.0956
20m 30s (- 1m 22s) (120 93%) 0.0972
21m 18s (- 0m 30s) (125 97%) 0.0963


In [ ]:
def evaluate(encoder, decoder, sentence):
    with torch.no_grad():
        input_tensor = tensorFromSentence(chinese_tokenizer, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(english_index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [16]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> 我曾與他的比賽網球
= i had a tennis match with him
< i felt a tennis match with him <EOS>

> 我举办了一场派对
= i threw a party
< i grow on me <EOS>

> 欢迎来到我们家
= welcome to our home
< welcome to our home <EOS>

> 我没有反对你
= im not disagreeing with you
< im not disagree ing with you <EOS>

> 該建築高一百公尺
= the building is one hundred meters high
< the day passes one s were ready <EOS>

> 我看见一个穿黑衣服的女人
= i saw a woman in black
< come down <EOS>

> 每个人都知道汤姆的法语很好
= everyone knows that toms french is good
< everyone knew tom was good at french <EOS>

> 你這個星期一直在做什麼
= what have you been doing this week
< what have you been doing this week <EOS>

> 為甚麼他不再跟我講笑話了
= why doesnt he tell me jokes anymore
< why he was wasting you and drinking <EOS>

> 突然開始下雨
= all of a sudden it began raining
< it suddenly started raining <EOS>



## show attention

In [17]:
def showAttention(input_sentence, output_words, attentions, filename):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # plt.show()
    fig.savefig(filename)


def evaluateAndShowAttention(input_sentence, filename):
    output_words, attentions = evaluate(encoder, decoder, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :], filename)


evaluateAndShowAttention('嗨。', "嗨attnMap.png")

evaluateAndShowAttention('你用跑的。', "你用跑的attnMap.png")

evaluateAndShowAttention('我已经起来了。', "我已经起来了attnMap.png")

evaluateAndShowAttention('静静的，别动。', "静静的别动attnMap.png")

input = 嗨。
output = hi where does your uncle name <EOS>
input = 你用跑的。
output = run <EOS>
input = 我已经起来了。
output = ive already watch my time <EOS>
input = 静静的，别动。
output = please leave me alone <EOS>


C:\Users\86183\AppData\Local\Temp\ipykernel_25312\902765783.py:8: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\86183\AppData\Local\Temp\ipykernel_25312\902765783.py:10: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
C:\Users\86183\AppData\Local\Temp\ipykernel_25312\902765783.py:17: UserWarning: Glyph 21992 (\N{CJK UNIFIED IDEOGRAPH-55E8}) missing from current font.
  fig.savefig(filename)
C:\Users\86183\AppData\Local\Temp\ipykernel_25312\902765783.py:17: UserWarning: Glyph 12290 (\N{IDEOGRAPHIC FULL STOP}) missing from current font.
  fig.savefig(filename)
C:\Users\86183\AppData\Local\Temp\ipykernel_25312\902765783.py:8: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks